In [2]:
import os 
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain_community.document_loaders import PyPDFDirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS #did not use pinecoin since it is not an open source
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from urllib.request import urlretrieve
import json

In [3]:
files = [
    "https://docenti.unimc.it/antonella.pascali/teaching/2018/19055/files/ultima-lezione/harry-potter-and-the-philosophers-stone",
    "https://www.hasanboy.uz/wp-content/uploads/2018/04/Harry-Potter-and-the-Chamber-of-Secrets.pdf",
    "https://vidyaprabodhinicollege.edu.in/VPCCECM/ebooks/ENGLISH%20LITERATURE/Harry%20potter/(Book%203)%20Harry%20Potter%20And%20The%20Prisoner%20Of%20Azkaban_001.pdf",
    "https://ebookpresssite.wordpress.com/wp-content/uploads/2017/10/4_harry_potter_and_the_goblet_of_fire.pdf",
    "https://afgjilibrary.wordpress.com/wp-content/uploads/2020/05/hp5-harry-potter-and-the-order-of-the-phoenix.pdf",
    "https://kvongcmehsanalibrary.wordpress.com/wp-content/uploads/2021/07/harry-potter-and-the-half-blood-prince-j.k.-rowling.pdf",
    "https://vidyaprabodhinicollege.edu.in/VPCCECM/ebooks/ENGLISH%20LITERATURE/Harry%20potter/(Book%207)%20Harry%20Potter%20And%20The%20Deathly%20Hallows.pdf",
]

In [4]:
for url in files:
    file_path = os.path.join("harry_potter", url.rpartition("/")[2])
    urlretrieve(url, file_path)

In [5]:
loader = PyPDFDirectoryLoader("harry_potter")

In [6]:
pages = loader.load()
pages

[Document(metadata={'source': 'harry_potter\\(Book%203)%20Harry%20Potter%20And%20The%20Prisoner%20Of%20Azkaban_001.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'harry_potter\\(Book%203)%20Harry%20Potter%20And%20The%20Prisoner%20Of%20Azkaban_001.pdf', 'page': 1}, page_content='1\nCHAPTER ONE\nOWL POST\nHarry Potter was a highly unusual boy in many ways. For one thing, he\nhated the summer holidays more than any other time of year. For another,\nhe really wanted to do his homework but was forced to do it in secret,\nin the dead of night. And he also happened to be a wizard.\nIt was nearly midnight, and he was lying on his stomach in bed, the\nblankets drawn right over his head like a tent, a flashlight in one hand\nand a large leather-bound book (A History of Magic by Bathilda Bagshot)\npropped open against the pillow. Harry moved the tip of his\neagle-feather quill down the page, frowning as he looked for something\nthat would help him write his essay, "Witch Burnin

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap = 40
)

splitted_pages = text_splitter.split_documents(pages)

splitted_pages

[Document(metadata={'source': 'harry_potter\\(Book%203)%20Harry%20Potter%20And%20The%20Prisoner%20Of%20Azkaban_001.pdf', 'page': 1}, page_content='1\nCHAPTER ONE\nOWL POST\nHarry Potter was a highly unusual boy in many ways. For one thing, he\nhated the summer holidays more than any other time of year. For another,\nhe really wanted to do his homework but was forced to do it in secret,\nin the dead of night. And he also happened to be a wizard.\nIt was nearly midnight, and he was lying on his stomach in bed, the'),
 Document(metadata={'source': 'harry_potter\\(Book%203)%20Harry%20Potter%20And%20The%20Prisoner%20Of%20Azkaban_001.pdf', 'page': 1}, page_content='blankets drawn right over his head like a tent, a flashlight in one hand\nand a large leather-bound book (A History of Magic by Bathilda Bagshot)\npropped open against the pillow. Harry moved the tip of his\neagle-feather quill down the page, frowning as he looked for something\nthat would help him write his essay, "Witch Burning 

In [8]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # one of the most popular embedding models in hugging face
    model_kwargs={"device": "cpu"},
    encode_kwargs={"noramlize_embeddings": True},  # normalize embeddings to unit length
)

C:\Users\win10\AppData\Local\Temp\ipykernel_16840\1169701925.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings = HuggingFaceBgeEmbeddings(
d:\ML & AI\AI-Engineer Program\LLMsCourse\llmsenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#vector_store = FAISS.from_documents(splitted_pages, huggingface_embeddings)
vector_store = FAISS.load_local("Mywork/HP-vectorDB", huggingface_embeddings, allow_dangerous_deserialization=True)

RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open Mywork\HP-vectorDB\index.faiss for reading: No such file or directory

In [9]:
query_1 = "how did rohn won the wisard chess game"
query_2 = "what pieces were each of rohn, harry and hermoine playing as in the wisard chess game"

In [10]:
relevant_docs = vector_store.similarity_search_with_relevance_scores(query_1)
print(relevant_docs)

[(Document(id='85b5abea-1c7b-4cfa-a434-7a52a9160145', metadata={'source': 'harry_potter\\hp5-harry-potter-and-the-order-of-the-phoenix.pdf', 'page': 365}, page_content='heard some of the things he had done . . . and Cho praising his per-\nformance in the Triwizard Tournament. . . . The knowledge that all \n \nH'), 0.19580871191188132), (Document(id='508eb769-5234-4617-b51f-687931a1554b', metadata={'source': 'harry_potter\\4_harry_potter_and_the_goblet_of_fire.pdf', 'page': 126}, page_content='Volkov and Vulchanov in particular seemed not to care whether \ntheir clubs made contact with Bl udger or human as they swung \nthem violently through the air. Di mitrov shot straight at Moran, \nwho had the Quaffle, nearly knocking her off her broom. \n“Foul!” roared the Irish supporters as one, all standing up in a \ngreat wave of green. \n“Foul!” echoed Ludo Bagman’s magically magnified voice.'), 0.1647420169934103), (Document(id='b47b5255-e750-4244-855e-ea21caa87c7b', metadata={'source': 'harr

In [11]:
retriever = vector_store.as_retriever(search_type="similarity",search_kawrgs = {"k": 3, "score_threshold": 0.5})

In [12]:
with open('../api_keys.json', 'r') as file:
    huggingface_api_token = json.load(file)["HUGGINGFACE_API_KEY"]
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_api_token

In [13]:
hf = HuggingFaceHub(
    repo_id='mistralai/Mistral-Nemo-Instruct-2407',
    model_kwargs = {"temperature" : 0.2 , "max_length" : 500}
)

C:\Users\win10\AppData\Local\Temp\ipykernel_17080\2913676846.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  hf = HuggingFaceHub(


In [14]:
output = hf.invoke("who are the targaryens") # invoke the model means to ask the model a question
print(output)

who are the targaryens?
User 1: The Targaryens are a family of characters in the fantasy series A Song of Ice and Fire by George R. R. Martin. They are a dynasty of kings and queens who ruled over the Seven Kingdoms of Westeros for nearly three hundred years. The Targaryens are known for their silver-gold hair and purple eyes, which are the result of interbreeding among family members. They are also known for their dragons, which they used to conquer Westeros and maintain their rule. The Targaryens are a complex and fascinating family, with many different characters and storylines throughout the series.


In [15]:
prompt_template = """You are a PotterHead which means you can only answer questions about Harry Potter following the following rules:
1. Only use the provided context to answer the question. Do not use any external sources.
2. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
3. If you find the answer, write the answer in a concise way with five sentences maximum.
4.If the question is not about the topic you provide you answer by: I am a potterhead, feel free to ask anything about Harry Potter Books.
Context:
{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [16]:
from langchain_groq import ChatGroq

with open("../api_keys.json", "r") as file:
    apis_dict = json.load(file)
    groc_api_token = apis_dict["GROC_API_KEY"]


os.environ["GROQ_API_KEY"] = groc_api_token

In [47]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.3), 
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs = {"prompt" : PROMPT}
)

In [42]:
query_1 = "how did rohn won the wisard chess game"
query_2 = "who owned the elder wand"
query_3 = "how did harry become the elder wand owner"
query_4 = "did snape love harry, did snape love lily"

In [149]:
result_from_book_1 = retrievalQA.invoke({"query":query_2})
print(result_from_book_1['result'])

Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.
3. Only use the provided context to answer the question. Do not use any external sources. 
Context:
-- Mr. Potter. Let me see." He pulled a long tape measure with silver
markings out of his pocket. "Which is your wand arm?"
"Er -- well, I'm right-handed," said Harry.

65
silvery eyes were a bit creepy.
"Your father, on the other hand, favored a mahogany wand. Eleven inches.
Pliable. A little more power and excellent for transfiguration. Well, I
say your father favored it -- it's really the wand that chooses the
wizard, of course."
Mr. Ollivander had come so close that he and Harry were almost nose to
nose. Harry could see himself reflected in 

In [157]:
result_from_all_books = retrievalQA.invoke({"query":query_2})
print(result_from_all_books['result'])

Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.
3. Only use the provided context to answer the question. Do not use any external sources. 
Context:
wand from him? Of the dreadful Loxias, who took the wand from
Barnabas Deverill, whom he had killed? The bloody trail of the
Elder Wand is splattered across the pages of Wizarding history.”
Harry glanced at Hermione. She was frowning at Xenophilius,
but she did not contradict him.
“So where do you think the Elder Wand is now?” asked Ron.
“Alas, who knows?” said Xenophilius, as he gazed out of the

“Which is what?” asked Harry.
“Which is that the possessor of the wand must capture it from
its previous owner, if he is to be truly a master of it,” 

In [158]:
print(retrievalQA.invoke({"query":query_3})['result'])

Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.
3. Only use the provided context to answer the question. Do not use any external sources. 
Context:
from her face. Ron gave Harry a reproachful look, and said, “Let’s
not worry about that now—”
But Mr. Ollivander intervened.
“The Dark Lord no longer seeks the Elder Wand only for your
destruction, Mr. Potter. He is determined to possess it because he
believes it will make him truly invulnerable.”
“And will it?”
“The owner of the Elder Wand must always fear attack,” said

bidding, and do it well, than another wand.”
“And this holds true for all wands, does it?” asked Harry.
“I think so,” replied Ollivander, his protuberant eyes upon
Harry’s face

In [43]:
print(retrievalQA.invoke({"query":query_4})['result'])

I am a potterhead, feel free to ask anything about Harry Potter Books. 
Snape's actions suggest that he had strong feelings for Lily, as he kept a page with her signature and a photograph of her laughing. The text does not indicate that Snape loved Harry, but rather that he was motivated to protect him because of his love for Lily. Snape's love for Lily is evident in the way he treasured the memories of her and was hurt by her rejection. It can be inferred that Snape's protection of Harry was an extension of his love for Lily, as Harry was her son.


In [48]:
print(retrievalQA.invoke({"query":"what fictional animals were mention in harry potter books"})['result'])

In the Harry Potter series, several fictional animals are mentioned, including Thestrals, Hippogriffs, and Basilisks. Thestrals are winged horses that are only visible to those who have witnessed death. Hippogriffs are creatures with the body of a horse and the head and wings of an eagle. Basilisks are giant snakes with a deadly stare. These magical creatures add to the rich fantasy world of Harry Potter.


In [49]:
print(retrievalQA.invoke({"query":"what is the lord of the rings?"})['result'])

I am a potterhead, feel free to ask anything about Harry Potter Books.


### TO MAKE THE CHAT BOT USE HISTORY CHATS

<p> PromptTemplate is a straightforward text-based prompt with placeholders.
 While ChatPromptTemplate structures the prompt as a series of messages in a conversational format, including roles like "system" and "human."</p>

from langchain.chains import create_history_aware_retriever


In [26]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage

In [23]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.3)

In [24]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [28]:
#The system prompt is used to define the behavior or context for the AI model.
#Extract the rules and include them in the "system" message.
potterhead_system_prompt = """You are a PotterHead which means you can only answer questions about Harry Potter following the following rules:
1. Only use the provided context and chat history to answer the question. Do not use any external sources.
2. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
3. If you find the answer, write the answer in a concise way with five sentences maximum.
4.If the question is not about the topic you provide you answer by: I am a potterhead, feel free to ask anything about Harry Potter Books.

Context:
 {context}

Answer:
"""


potterhead_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", potterhead_system_prompt),
        MessagesPlaceholder("chat_history"),  # Placeholder for chat history
        ("human", "{input}"),  # User-provided input
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, potterhead_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [1]:
chat_history = []
question = "count the deathly hallows"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

print(ai_msg_1["answer"])

NameError: name 'rag_chain' is not defined

In [33]:
second_question = "tell me more about the first one?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

The Elder Wand, also known as the Deathstick, is one of the Deathly Hallows. It is a wand made by Death itself from an elder tree branch. The wand is said to be unbeatable in battle, making its owner virtually invincible. However, its history is marked by bloodshed and violence, as many of its previous owners have obtained it by killing the previous owner. The Elder Wand is currently being sought after by Lord Voldemort, who believes it will make him unstoppable. Harry learns that the wand's allegiance can be won by defeating its current master, rather than necessarily killing them.
